In [8]:
!pip -q install langchain langchain_community pypdf fastembed chromadb


In [ ]:
!pip install streamlit


In [10]:
!pip install langchain 

In [11]:
!pip install langchain_openai

In [12]:
!pip install pypdf 

In [13]:
!pip install langchain_community

In [14]:

import streamlit as st
from langchain_core.messages import AIMessage, HumanMessage
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from openai import OpenAI

In [ ]:
# import the question-answering chain and Huggingface Hub LLM
from langchain.llms import HuggingFaceHub
import os


 




In [22]:
# import PDF reader
from langchain_community.document_loaders.pdf import PyPDFLoader

# load the document
# Source credits: https://ncert.nic.in/textbook/pdf/lekl101.pdf
loader = PyPDFLoader('C:/Users/PC/Desktop/VS code File/chatbot_strealit/src/guide.pdf')
data = loader.load()

In [23]:
# import text splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
chunk_size = 512,
chunk_overlap = 0,
)

chunks = text_splitter.split_documents(data)


In [24]:

from langchain_community.embeddings.fastembed import FastEmbedEmbeddings

model_name = "thenlper/gte-large"
embedding_model = FastEmbedEmbeddings(model_name="thenlper/gte-large")


 

In [25]:
 
from langchain.vectorstores import Chroma

# initialize the vector store (save to disk)
db = Chroma.from_documents(chunks, embedding_model, persist_directory="./chroma_db")

 
# Let's define the query, since we are going to use it multiple times.
query = "Give me the main comptetence of mohamed"

In [26]:
# retrieve from vector db (load from disk) with query
retrieved_docs = db.similarity_search(query)
print(retrieved_docs[0].page_content)


 
# initialize the retriever
retriever = db.as_retriever(
search_type="mmr", #similarity
search_kwargs={'k': 4}
)



Il est important de noter les points suivants :
• La puissance photovoltaique maximale est la valeur minimale entre la puissance 
crête totale des modules photovoltaïques et la somme des puissances nominales 
des onduleurs.
• Plusieurs socles de prises 230V 16A peuvent être alimentés sur le même circuit. Le 
tableau ci-dessus est donné pour un circuit et non pas pour une seule prise 230V.
• Une prise de courant 230V 16A 2P+T n’est pas dimensionnée pour faire transiter un


In [27]:
 
# define the llm
llm = HuggingFaceHub(repo_id="mistralai/Mistral-7B-Instruct-v0.2",
model_kwargs={
"temperature":0.1,
"max_new_tokens":512,
"return_full_text":False,
"repetition_penalty":1.1,
"top_p":0.9
})



C:\Users\PC\AppData\Local\Temp\ipykernel_21288\3703919865.py:2: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm = HuggingFaceHub(repo_id="mistralai/Mistral-7B-Instruct-v0.2",


In [28]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

template = """
<s>[INST]
You are an AI Assistant that follows instructions extremely well.
Please be truthful and give direct answers. Please tell 'I don't know' if user query is not in CONTEXT
[/INST]
CONTEXT: {context}
</s>
[INST]
{query}
[/INST]
"""

prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

chain = (
{"context": retriever, "query": RunnablePassthrough()}
| prompt
| llm
| output_parser
)


In [29]:
response = chain.invoke(query)
response



'Based on the context provided, Mohamed is not mentioned in any of the documents. Therefore, I cannot provide information about his main competences.'

In [30]:

response3 = chain.invoke("donne moi c'est quoi le resme in 3 line")
print(response3)

The document discusses electrical consumption and solar current injection limitations for electrical circuits. It includes examples of wiring configurations and safety considerations for photovoltaic systems. The manufacturer is required to perform a risk analysis for atmospheric conditions during installation.


In [31]:
response3 = chain.invoke("can you read the first photo")
print(response3)

I'm an AI language model and cannot directly read or process images. However, based on the context provided, the first document appears to discuss examples of solar photovoltaic kits, specifically those for wall mounting and ground installation. The text also mentions "fig.1" which could potentially refer to a figure or image in the document, but without access to the actual image, I can't provide further details.
